In [1]:
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen
import re
import pandas as pd
import requests
from requests import get
import re

# Pulling all links off of the search pages (up to 3000) and putting them in a dataframe to scrape.

In [2]:
url_template="http://www.indeed.com/jobs?q=%22Machine+Learning+Engineer%22&l=Cupertino%2C+CA&start={}"
max_results=250
Linkdf=[]

for start in range(0, max_results, 7):
    url=url_template.format(start)
    html=requests.get(url)
    soup=BeautifulSoup(html.content,'html.parser', from_encoding="utf-8")
    
    #for each in soup.find_all(a_="href"):
    page_links=soup.find_all('a',{'href':re.compile("/rc/")})
    for items in page_links:
        Linkdf.append(items['href'])   

In [3]:
#Sanity Check
len(Linkdf)
#print(Linkdf)

325

In [4]:
#This code allows the code to display the full website instead of truncating
pd.set_option('display.max_colwidth',-1)

#Moving it to a data frame
data = {'links':Linkdf}
df = pd.DataFrame(data, columns=['links'])

#append indeed.com to the front of each
df['Web'] = 'https://www.indeed.com'
df['URL'] = df.Web.str.cat(df.links)

#pull out just a list of the websites.
websites=list(df['URL'])

#Sanity Check
#print(websites)
len(websites)

325

In [5]:
websites1=set(websites)

In [6]:
len(websites1)

107

# Looping through websites....

In [7]:
Title=[]
Descriptions=[]
Location=[]
FullDescriptions=[]

for url in websites1:
    response=get(url)
    soup=BeautifulSoup(response.text,'html.parser')
    
    description_containers= soup.find(class_='jobsearch-jobDescriptionText')
    title_containers=soup.find('h3')
    try:
        location_containers=soup.find('',{'class':'jobsearch-CompanyInfoWithoutHeaderImage'}).find_all('div')[-1]
    except:
        location_containers='None Found'
    
    job_descriptions=str(description_containers)
    job_title=str(title_containers.text)
    try:
        locations=str(location_containers.text)
    except AttributeError:
        locations = 'None Found'
    try:
        full_descriptions = str(description_containers.text)
    except AttributeError:
        full_descriptions= 'None Found'
    
    Descriptions.append(job_descriptions)
    Title.append(job_title)
    Location.append(locations)
    FullDescriptions.append(full_descriptions)    

# Extracting what we want from the Descriptions Column

In [8]:
#'Location' left in for sanity check. Should be removed once code is confirmed to work
Descriptions_df = pd.DataFrame(columns = ['Title', 'Location','City', 'State', 'Zip', 'Country', 'Qualifications', 'Skills', 'Responsibilities', 'Education', 'Requirement', 'FullDescriptions'])
Country = ['US', 'USA', 'United States', 'United States of Americal']
States = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA',
          'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND',
          'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

for index, element in enumerate(Descriptions):
    soup=BeautifulSoup(element,'lxml')
    for values in list(Descriptions_df):
        temp_tag = soup.find('b', text=re.compile(values))
        try:
            ul_tag = temp_tag.find_next('ul')
            Descriptions_df.at[index,values] = ul_tag.text
        except AttributeError:
            Descriptions_df.at[index,values]="None Found"
        Descriptions_df.at[index,"Title"]=Title[index]
        Descriptions_df.at[index,"Location"]=Location[index]
        Descriptions_df.at[index,"FullDescriptions"]=FullDescriptions[index]
        words = '|'.join(Country)
        temp = re.findall(words, Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"Country"] = temp[0]
        words = '|'.join(States)
        temp = re.findall(words, Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"State"] = temp[0]
        temp = re.findall(r'\d+', Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"Zip"] = temp[0]  
            
        temp = re.findall(r'[\w w]+,', Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"City"] = re.sub(',', '', temp[0])


In [9]:
Descriptions_df

Title  \
0    Sr. Machine Learning Engineer – Apple Maps Search   
1    Machine Learning Engineer                           
2    Senior Data Scientist / Machine Learning Engineer   
3    Machine Learning Engineer / Forecasting             
4    Machine Learning Engineer                           
..                         ...                           
102  Sr. Machine Learning Engineer - News                
103  Senior Machine Learning Engineer                    
104  Principal Machine Learning Engineer                 
105  GPGPU Machine Learning Engineer                     
106  Machine Learning Engineer                           

                         Location                City State         Zip  \
0    Santa Clara Valley, CA 95014  Santa Clara Valley  CA    95014        
1    Redwood City, CA              Redwood City        CA    None Found   
2    Redwood City, CA 94063        Redwood City        CA    94063        
3    Mountain View, CA             Mountain View       CA    None Found   
4    Palo Alto, CA                 Palo Alto           CA    None Found   
..             ...                       ...           ..           ...   
102  Santa Clara Valley, CA 95014  Santa Clara Valley  CA    95014        
103  Sunnyvale, CA                 Sunnyvale           CA    None Found   
104  Sunnyvale, CA 94087           Sunnyvale           CA    94087        
105  Santa Clara, CA 95052         Santa Clara         CA    95052        
106  Santa Clara, CA               Santa Clara         CA    None Found   

        Country  \
0    None Found   
1    None Found   
2    None Found   
3    None Found   
4    None Found   
..          ...   
102  None Found   
103  None Found   
104  None Found   
105  None Found   
106  None Found   

                                                                                                                                                                                                                                                                                                                                                                                                                                        Qualifications  \
0    None Found                                                                                                                                                                                                                                                                                                                                                                                                                                          
1    \nExpertise in at least one of: C++, Python, Java, Go (two or more preferred).\n3+ years of experience working with TensorFlow, PyTorch, or other mainstream ML framework.\n1+ years of experience with deep learning on production-grade systems at scale.\nExcellent interpersonal, leadership and communication skills, including the ability to explain complex technical concepts in simple terms and desire to work closely with end-users.   
2    None Found                                                                                                                                                                                                                                                                                                                                                                                                                                          
3    None Found                                                                                                                                                                                                                                                                                                                                                                                                                                          
4    None Fo

# Move to csv

In [10]:
Descriptions_df.to_csv('Descriptions_df_MLE_Cupertino.csv')